In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statistics
import warnings
from matplotlib import rc
from sklearn.preprocessing import LabelEncoder

plt.rcParams['axes.unicode_minus'] = False
rc('font', family='Malgun Gothic')
%matplotlib inline


warnings.filterwarnings('ignore')

# Train

In [2]:
df = pd.read_csv("./Dataset/train.csv")
df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.164979,Standard


In [3]:
# 데이터 컬럼 네임 정리
df = df.rename(columns={"ID": "거래내역",
                   "Customer_ID": "고객ID",
                   "Month": "월",
                   "Name": "이름",
                   "Age": "나이",
                   "SSN": "식별번호",
                   "Occupation": "직업",
                   "Annual_Income": "연수입",
                   "Monthly_Inhand_Salary": "월급",
                   "Num_Bank_Accounts": "계좌 수",
                   "Num_Credit_Card": "신용카드 수",
                   "Interest_Rate": "이자율",
                   "Num_of_Loan": "대출수",
                   "Type_of_Loan": "대출종류",
                   "Delay_from_due_date": "연체일수",
                   "Num_of_Delayed_Payment": "연체 지불횟수",
                   "Changed_Credit_Limit": "카드 한도 변경",
                   "Num_Credit_Inquiries": "신용카드 조회 횟수",
                   "Credit_Mix": "신용예측 정도",
                   "Outstanding_Debt": "남아있는 부채",
                   "Credit_Utilization_Ratio": "신용카드 이용률 비율",
                   "Credit_History_Age": "계좌개설 나이", # 부모님의 금융에 대한 관삼을 유추할 수 있음
                   "Payment_of_Min_Amount": "최소금액 지불 여부",
                   "Total_EMI_per_month": "월별 할부 지불액",
                   "Amount_invested_monthly": "월별 투자금액",
                   "Payment_Behaviour": "고객의 소비패턴",
                   "Monthly_Balance": "월별 고객 잔액",
                   "Credit_Score": "신용점수",
                   })

In [4]:
def month_to_number(month):
    month_dict = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4,
        'May': 5, 'June': 6, 'July': 7, 'August': 8,
        'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    return month_dict[month]

df['월'] = df['월'].apply(month_to_number)

# 나이
- 특정 나이(57세) 이상부터 value_counts()확인 결과 분포가 이상치로 확인
- 57세 이상 -1로 변환 후 해당 값을 고객ID별 최빈값으로 변환
- 최빈값 이유 : 1~8개월에 따라 나이의 변동이 지극히 작기 때문

In [5]:
df.columns

Index(['거래내역', '고객ID', '월', '이름', '나이', '식별번호', '직업', '연수입', '월급', '계좌 수',
       '신용카드 수', '이자율', '대출수', '대출종류', '연체일수', '연체 지불횟수', '카드 한도 변경',
       '신용카드 조회 횟수', '신용예측 정도', '남아있는 부채', '신용카드 이용률 비율', '계좌개설 나이',
       '최소금액 지불 여부', '월별 할부 지불액', '월별 투자금액', '고객의 소비패턴', '월별 고객 잔액', '신용점수'],
      dtype='object')

In [6]:
df['나이'] = df['나이'].str.replace("_", "")
df['나이'] = df['나이'].astype('int')
df.loc[df['나이']>=57, "나이"] = -1
df.loc[df['나이']<0, "나이"] = -1

In [7]:
list_id = df['고객ID'].unique()

for id in list_id:
    index_list = df[(df['고객ID']==id) & (df['나이'] == -1)].index
    df.loc[index_list, "나이"] = statistics.mode(df[(df["고객ID"]==id) & (df['나이']!=-1)]['나이'])

# 연체 지불횟수 전처리
- "_"문자열 삭제
- Nan값 -1으로 대체
- int형태로 변환 후 고객ID별로 -1로 채운 Nan값을 최빈값으로 대체

In [10]:
df['연체 지불횟수'] = df['연체 지불횟수'].str.replace("_", "")
df['연체 지불횟수'] = df['연체 지불횟수'].fillna(-1)
df['연체 지불횟수'] = df['연체 지불횟수'].astype('int')
df.loc[df['연체 지불횟수']>25, '연체 지불횟수'] = -1

In [11]:
list_id = df['고객ID'].unique()

for id in list_id:
    try:
        index_list = df[(df['고객ID']==id) & (df['연체 지불횟수'] == -1)].index
        df.loc[index_list, "연체 지불횟수"] = statistics.mode(df[(df["고객ID"]==id) & (df['연체 지불횟수']!=-1)]['연체 지불횟수'])
    except:
        df.loc[index_list, "연체 지불횟수"] = 0

# 카드 한도 변경
- "_"특수문자 제거
- 숫자가 아닌 데이터 NaN값으로 대체
- 빈값을 -1로 대체
- 데이터 타입을 float으로 변경
- -1값을 고객ID별로 

In [12]:
df['카드 한도 변경'] = df['카드 한도 변경'].str.replace("_", "")
df['카드 한도 변경'] = pd.to_numeric(df['카드 한도 변경'], errors='coerce')
df['카드 한도 변경'] = df['카드 한도 변경'].fillna(-1)
df['카드 한도 변경'] = df['카드 한도 변경'].astype("float")

In [13]:
for id in list_id:
    index_list = df[(df['고객ID']==id) & (df['카드 한도 변경'] == -1)].index
    df.loc[index_list, "카드 한도 변경"] = np.average(df[(df["고객ID"]==id) & (df['카드 한도 변경']!=-1)]['카드 한도 변경'])

# 월별 투자금액
- '_' 지우고, float 타입 변환
- 월별 투자금액이 10000인 값 -> 결측치로 둠

In [14]:
df['월별 투자금액'] = df['월별 투자금액'].str.replace('_', '')
df['월별 투자금액'] = df['월별 투자금액'].astype('float')

df['월별 투자금액'] = np.where(df['월별 투자금액'] == 10000, np.nan, df['월별 투자금액'])

df['월별 투자금액'] = df['월별 투자금액'].fillna(df.groupby(['고객ID'])['월별 투자금액'].transform('median'))

# 남아있는 부채 전처리

In [15]:
df['남아있는 부채'] = df['남아있는 부채'].str.replace('_', '')
df['남아있는 부채'] = df['남아있는 부채'].astype('float')

# 신용카드 조회 횟수
- 신용카드 조회 횟수 20이상의 경우 각 값의 수치가 낮고, 이상치로 판단
- 20회 이상의 신용카드 조회 횟수를 -1로 변경
- 이후 고객ID별로 -1 즉 이상치 데이터를 최빈값으로 변경

In [16]:
df.loc[df['신용카드 조회 횟수'] >= 20, '신용카드 조회 횟수'] = -1
df['신용카드 조회 횟수'] = df["신용카드 조회 횟수"].fillna(-1)

for id in list_id:
    index_list = df[(df['고객ID']==id) & (df['신용카드 조회 횟수'] == -1)].index
    df.loc[index_list, "신용카드 조회 횟수"] = statistics.mode(df[(df["고객ID"]==id) & (df['신용카드 조회 횟수']!=-1)]['신용카드 조회 횟수'])

# 신용예측 정도
- 고객ID별 탐색 결과 모든 신용예측 정도가 "_" 결측값으로 입력된 경우는 없음
- 결측값을 -1로 변경 후 해당 고객ID별 최빈값을 입력

In [17]:
df.loc[df['신용예측 정도']=="_", "신용예측 정도"] = -1
df['신용예측 정도'] = df['신용예측 정도'].fillna(-1)

for id in list_id:
    index_list = df[(df['고객ID']==id) & (df['신용예측 정도'] == -1)].index
    df.loc[index_list, "신용예측 정도"] = statistics.mode(df[(df["고객ID"]==id) & (df['신용예측 정도']!=-1)]['신용예측 정도'])

# 고객의 소비패턴
- 결측치 없음
- 고객의 소비패턴이 '!@9#%8'인 값 존재 -> 고객ID별로 소비패턴 최빈값으로 대체

In [18]:
idx_list = df[df['고객의 소비패턴'] == '!@9#%8'].index

for idx in idx_list:
  id = df.loc[idx, '고객ID']
  df.loc[idx, '고객의 소비패턴'] = statistics.mode(df[(df['고객ID']==id) & (df['고객의 소비패턴']!= '!@9#%8')]['고객의 소비패턴'])

# 계좌 수 전처리
- 결측치 처리
- 10개 초과 개수, 음수는 이상치로 처리
- 고객 ID별로 nan값은 중앙값으로 대체

In [19]:
df['계좌 수'] = np.where(df['계좌 수'] > 10, np.nan,df['계좌 수'])
df['계좌 수'] = np.where(df['계좌 수'] < 0, np.nan,df['계좌 수'])
df['계좌 수'] = df['계좌 수'].fillna(df.groupby(['고객ID'])['계좌 수'].transform('median')) # 계좌가 없는데 신용카드 존재

# 신용카드 수 전처리
- 결측치 처리
- 10개 초과 개수, 음수는 이상치로 처리
- 고객 ID별로 nan값은 중앙값으로 대체

In [20]:
df['신용카드 수'] = np.where(df['신용카드 수'] > 10, np.nan,df['신용카드 수'])
df['신용카드 수'] = np.where(df['신용카드 수'] < 0, np.nan,df['신용카드 수'])
df['신용카드 수'] = df['신용카드 수'].fillna(df.groupby(['고객ID'])['신용카드 수'].transform('median'))

# 최소금액 지불 여부 전처리
- '최소금액 지불 여부' 값이 'NM'인 경우, 각 고객ID 그룹 내에서 가장 빈도가 높은 값으로 대체
- YES를 1 NO를 0으로 변환+ 타입을 int로 바꾸기

In [21]:
def replace_nm_with_mode(group):
    nm_mask = group == 'NM'
    mode_value = group[~nm_mask].mode().iloc[0] if not group[~nm_mask].empty else 'NM'
    group[nm_mask] = mode_value
    return group

df['최소금액 지불 여부'] = df.groupby('고객ID')['최소금액 지불 여부'].transform(replace_nm_with_mode)
df['최소금액 지불 여부'] = np.where(df['최소금액 지불 여부'] == 'Yes', 1, 0)
df['최소금액 지불 여부'] = df['최소금액 지불 여부'].astype('int')

# 대출수 전처리
- '_' 문자열 삭제
- int 형태로 변환
- 음수 값을 최빈값으로 바꾸는 함수 정의
- '대출수' 컬럼의 음수 값을 최빈값으로 바꾸기
- 10 이상인 값을 최빈값으로 바꾸는 함수
- '대출수' 컬럼의 10 이상인 값을 최빈값으로 바꾸기

In [22]:
df['대출수'] = df['대출수'].str.replace('_', '')
df['대출수'] = df['대출수'].astype(int)

def replace_1(series):
    mode_value = series.mode().iloc[0]
    return series.apply(lambda x: mode_value if x < 0 else x)

df['대출수'] = df.groupby('고객ID')['대출수'].transform(replace_1)

def replace_2(series):
    mode_value = series.mode().iloc[0]
    return series.apply(lambda x: mode_value if x > 10 else x)

df['대출수'] = df.groupby('고객ID')['대출수'].transform(replace_2)

# 월별 할부 지불액
- 결측치 없음
- 10000 이상인 값 2806개 존재 -> 극단적인 이상치로 봄
- 10000 이상인 값 -> 고객ID별 중앙값으로 대체

In [23]:
idx = df[df['월별 할부 지불액'] >= 10000].index
df.loc[df[df['월별 할부 지불액'] >= 10000].index, '월별 할부 지불액'] = np.NaN
df['월별 할부 지불액'] = df['월별 할부 지불액'].fillna(df.groupby(['고객ID'])['월별 할부 지불액'].transform('median'))

# 월별 고객 잔액
- 월별 고객 잔액이 '__-333333333333333333333333333__' 인 값 -> 결측치로 둠
- float 타입 변환
- 결측치 처리 -> 고객ID 별 월별 고객 잔액의 중앙값으로 대체

In [24]:
df['월별 고객 잔액'] = np.where(df['월별 고객 잔액'] == '__-333333333333333333333333333__', np.nan, df['월별 고객 잔액'])
df['월별 고객 잔액'] = df['월별 고객 잔액'].astype('float')
df['월별 고객 잔액'] = df['월별 고객 잔액'].fillna(df.groupby(['고객ID'])['월별 고객 잔액'].transform('median'))

# 이자율
- 이자율 이상치를 최빈값으로 바꾸기
- 이자율' 컬럼의 이상치를 최빈값으로 바꾸기

In [25]:
def replace_interest(series):
    mode_value = series.mode().iloc[0]
    return series.apply(lambda x: mode_value if x > 34 else x)

df['이자율'] = df.groupby('고객ID')['이자율'].transform(replace_interest)

# 연체일수
- 연체일수 음수를 0으로 변경

In [26]:
df['연체일수'] = np.where(df['연체일수'] < 0, 0, df['연체일수'])

# 대출 종류
- 대출종류 컬럼 값에서 'and'와 공백 제거
- 대출종류 고유리스트 생성
- 대출수가 0인 대출 종류를 None으로 변경
- name_list를 각각 열에 추가하고, 각 대출종류별 등장 횟수를 업데이트

In [27]:
df['대출종류'] = np.where(df['대출수'] == 0, 'None', df['대출종류'])

df['대출종류'] = df['대출종류'].str.replace('and', '').str.replace(' ', '')

name_list = []
test_list = list(df['대출종류'].unique())
for word in test_list:
    list_name = word.split(",")
    for name in list_name:
        if name not in name_list:
            name_list.append(name)

for name in name_list:
    df[name] = 0
for idx, row in df.iterrows():
    types = row['대출종류'].split(',')
    for t in types:
        df.at[idx, t] += 1

# 월급 
- Nan값을 근접한 값으로 대체

In [28]:
df['월급'] = df.groupby('고객ID')['월급'].fillna(method='bfill')
df['월급'] = df.groupby('고객ID')['월급'].fillna(method='ffill')

# 연수입

In [29]:
# 연수입 전처리
#"_"문자열 제거하기
# 비어있는 NaN값을 해당 하는 고객ID에서 최빈값으로 바꾸기
# float형으로 변경
# # 고객 ID별로 연수입이 같아야 하는데 다르게 나오는 이상치 구해서 최빈값으로 바꾸기

In [30]:
from statistics import mode

df['연수입']=df['연수입'].str.replace("_","")
df['연수입']=df['연수입'].astype('float')
df['연수입'] = df['연수입'].fillna(df.groupby(['고객ID'])['연수입'].transform('median'))
result_df = df.groupby('고객ID')['연수입'].agg(최빈값=pd.Series.mode, 평균값='mean').reset_index()
result_df['최종값'] = result_df.apply(lambda row: row['최빈값'] if row['최빈값'] == row['평균값'] else row['최빈값'], axis=1)
df['연수입'] = df.groupby('고객ID')['연수입'].transform(lambda group: mode(group))

# 직업
- "_" 고객 ID별 최빈값으로 변경

In [31]:
idx_list = df[df['직업'] == '_______'].index
for idx in idx_list:
  id = df.loc[idx, '고객ID']
  df.loc[idx, '직업'] = statistics.mode(df[(df['고객ID']==id) & (df['직업']!= '_______')]['직업'])

# 월별 할부 지불액
- 소수점 뒷자리가 000... 이면 이상치
- 소수점 뒷자리 추출
- 소수점 뒷자리가 000... 이면 결측치로 변환
- 0.1 -> 0
- 고객ID별로 앞에 있는 값으로 대체
- 위에서 결측치 대체 안 된 행은 고객ID별로 뒤에 있는 값으로 대체
- 0 -> 0.1

In [32]:
df['월별 할부 지불액'] = np.where(df['월별 할부 지불액'] == 0, df['월별 할부 지불액'] + 0.1, df['월별 할부 지불액'])
df['월별 할부 지불액 소수부분'] = df['월별 할부 지불액'].astype('str').str.split('.').str[1]
df['월별 할부 지불액'] = np.where(df['월별 할부 지불액 소수부분'] == '0', np.nan, df['월별 할부 지불액'])
df['월별 할부 지불액'] = np.where(df['월별 할부 지불액'] == 0.1, 0, df['월별 할부 지불액'])
df['월별 할부 지불액'] = df.groupby('고객ID')['월별 할부 지불액'].fillna(method = 'ffill')
df['월별 할부 지불액'] = df.groupby('고객ID')['월별 할부 지불액'].fillna(method = 'bfill')
df.drop(columns = ['월별 할부 지불액 소수부분'], inplace = True)

# 카드 발급 날짜 이후

In [33]:
def age_to_months(age_str):
    if pd.isnull(age_str) or age_str == 'NM':
        return None
    years, months = age_str.split(' Years and ')
    months = months.split(' Months')[0]
    total_months = int(years) * 12 + int(months)
    return total_months

# 'Credit_History_Age' 컬럼 값을 숫자로 변환하여 새로운 컬럼 'Months'에 저장
df['계좌개설 나이'] = df['계좌개설 나이'].apply(age_to_months)
# DataFrame에서 '신용카드 발급 이후 날짜' 컬럼에 대해 보간, 뒤채우기, 앞채우기 수행
df['계좌개설 나이'] = df.groupby('고객ID')['계좌개설 나이'].apply(lambda x: x.interpolate().bfill().ffill())
# DataFrame을 '고객ID'로 그룹화하여 8개씩 묶기
grouped = df.groupby('고객ID')
# 중복된 값이 나오면 뒤에 나오는 값이 +1이 되도록 처리하는 함수 정의
def adjust_duplicates(group):
    diff = group.diff()
    return group + (diff == 0).cumsum()
# '신용카드 발급 이후 날짜' 컬럼에 대해 중복 조정 적용
df['계좌개설 나이'] = grouped['계좌개설 나이'].apply(adjust_duplicates)
# DataFrame의 인덱스 재설정
df.reset_index(drop=True, inplace=True)

In [38]:
df.drop(columns=['거래내역', '이름', '식별번호', '대출종류'], inplace=True)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   고객ID                   100000 non-null  object 
 1   월                      100000 non-null  int64  
 2   나이                     100000 non-null  int32  
 3   직업                     100000 non-null  object 
 4   연수입                    100000 non-null  float64
 5   월급                     100000 non-null  float64
 6   계좌 수                   100000 non-null  float64
 7   신용카드 수                 100000 non-null  float64
 8   이자율                    100000 non-null  int64  
 9   대출수                    100000 non-null  int64  
 10  연체일수                   100000 non-null  int64  
 11  연체 지불횟수                100000 non-null  int32  
 12  카드 한도 변경               100000 non-null  float64
 13  신용카드 조회 횟수             100000 non-null  float64
 14  신용예측 정도                100000 non-nul

In [48]:
le = LabelEncoder()

df['직업'] = le.fit_transform(df['직업'])
df['신용예측 정도'] = le.fit_transform(df['신용예측 정도'])
df['고객의 소비패턴'] = le.fit_transform(df['고객의 소비패턴'])

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   고객ID                   100000 non-null  object 
 1   월                      100000 non-null  int64  
 2   나이                     100000 non-null  int32  
 3   직업                     100000 non-null  int32  
 4   연수입                    100000 non-null  float64
 5   월급                     100000 non-null  float64
 6   계좌 수                   100000 non-null  float64
 7   신용카드 수                 100000 non-null  float64
 8   이자율                    100000 non-null  int64  
 9   대출수                    100000 non-null  int64  
 10  연체일수                   100000 non-null  int64  
 11  연체 지불횟수                100000 non-null  int32  
 12  카드 한도 변경               100000 non-null  float64
 13  신용카드 조회 횟수             100000 non-null  float64
 14  신용예측 정도                100000 non-nul

In [50]:
df.to_csv("./Dataset/train_preprocessing.csv")

In [2]:
df = pd.read_csv('./Dataset/train_preprocessing.csv')
df.drop(columns='Unnamed: 0', inplace=True)

In [3]:
df['소비금액'] = df['월급'] - df['월별 할부 지불액'] - df['월별 투자금액'] - df['월별 고객 잔액']
df['이자금액'] = df['남아있는 부채'] * (df['이자율']/100)
df['연체기간'] = df['연체일수'] * df['대출수']
df['부채 대 소득 비율'] = df['남아있는 부채'] / df['월급']

In [4]:
def get_age_group(age):
    if age < 20:
        return '10대'
    elif age < 30:
        return '20대'
    elif age < 40:
        return '30대'
    elif age < 50:
        return '40대'
    else:
        return '50대'
# '연령대' 열 추가
df['연령대'] = df['나이'].apply(get_age_group)
df['연령대'].value_counts()

30대    29085
20대    28826
40대    22383
10대    11177
50대     8529
Name: 연령대, dtype: int64

In [31]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=13)
clusters = kmeans.fit(df[['남아있는 부채', '이자율', '신용예측 정도', '계좌개설 나이', '연체일수', '카드 한도 변경']])


df['cluster'] = clusters.labels_

In [40]:
df.rename(columns={'연체일수': '평균 연체일수', '연체 지불횟수': '평균 연체횟수', '이자율': '신용카드 이자율', '월별 고객 잔액': '월 계좌 잔액'}, inplace=True)

In [41]:
df_for_50 = df.drop(columns=['월급', '월별 할부 지불액', '월별 투자금액', '월 계좌 잔액', '남아있는 부채', '신용카드 이자율', '평균 연체일수', '평균 연체횟수'])

In [42]:
# 소비패턴 = High spent / Low spent
df_for_50['소비패턴'] = np.where((df_for_50['고객의 소비패턴'] == 0) | (df_for_50['고객의 소비패턴'] == 1) | (df_for_50['고객의 소비패턴'] == 2), 'High', 'Low')

In [43]:
le = LabelEncoder()

df_for_50['소비패턴'] = le.fit_transform(df_for_50['소비패턴'])

In [44]:
df_for_50.columns

Index(['고객ID', '월', '나이', '직업', '연수입', '계좌 수', '신용카드 수', '대출수', '카드 한도 변경',
       '신용카드 조회 횟수', '신용예측 정도', '신용카드 이용률 비율', '계좌개설 나이', '최소금액 지불 여부',
       '고객의 소비패턴', '신용점수', 'AutoLoan', 'Credit-BuilderLoan', 'PersonalLoan',
       'HomeEquityLoan', 'NotSpecified', 'None', 'MortgageLoan', 'StudentLoan',
       'DebtConsolidationLoan', 'PaydayLoan', '소비금액', '이자금액', '연체기간',
       '부채 대 소득 비율', '연령대', 'cluster', '소비패턴'],
      dtype='object')

In [46]:
df_for_50 = df_for_50.rename(columns={"연수입" : "연소득", "신용예측 정도" : "신용혼합도", '카드 한도 변경' : "카드 한도 변경률", '계좌개설 나이' : "계좌개설 경과기간(월)", '고객의 소비패턴' : "대출 상환 성향", '신용점수' : '신용등급'})

In [48]:
df_for_50.columns

Index(['고객ID', '월', '나이', '직업', '연소득', '계좌 수', '신용카드 수', '대출수', '카드 한도 변경률',
       '신용카드 조회 횟수', '신용혼합도', '신용카드 이용률 비율', '계좌개설 경과기간(월)', '최소금액 지불 여부',
       '대출 상환 성향', '신용등급', 'AutoLoan', 'Credit-BuilderLoan', 'PersonalLoan',
       'HomeEquityLoan', 'NotSpecified', 'None', 'MortgageLoan', 'StudentLoan',
       'DebtConsolidationLoan', 'PaydayLoan', '소비금액', '이자금액', '연체기간',
       '부채 대 소득 비율', '연령대', 'cluster', '소비패턴'],
      dtype='object')

In [49]:
df_for_50.to_csv("./Dataset/df_for_50.csv", index=False)

In [51]:
df_for_46 = df_for_50[df_for_50['나이']>=46]
df_for_46.to_csv("./Dataset/df_for_46.csv", index=False)

In [52]:
df_for = df_for_50[df_for_50['나이']<46]
df_for.to_csv("./Dataset/df_for.csv", index=False)

In [5]:
df = pd.read_csv("./Dataset/df_for.csv")
df.columns

Index(['고객ID', '월', '나이', '직업', '연소득', '계좌 수', '신용카드 수', '대출수', '카드 한도 변경률',
       '신용카드 조회 횟수', '신용혼합도', '신용카드 이용률 비율', '계좌개설 경과기간(월)', '최소금액 지불 여부',
       '대출 상환 성향', '신용등급', 'AutoLoan', 'Credit-BuilderLoan', 'PersonalLoan',
       'HomeEquityLoan', 'NotSpecified', 'None', 'MortgageLoan', 'StudentLoan',
       'DebtConsolidationLoan', 'PaydayLoan', '소비금액', '이자금액', '연체기간',
       '부채 대 소득 비율', '연령대', 'cluster', '소비패턴'],
      dtype='object')

In [6]:
df['소비패턴']

0        0
1        1
2        1
3        1
4        0
        ..
85703    0
85704    0
85705    0
85706    1
85707    0
Name: 소비패턴, Length: 85708, dtype: int64

In [7]:
df[df['소비패턴']==0]['소비금액'].describe()

count    38739.000000
mean      4163.836494
std       2801.378401
min         27.600625
25%       2042.087750
50%       3570.472500
75%       5701.326000
max      13444.170000
Name: 소비금액, dtype: float64

In [8]:
df[df['소비패턴']==1]['소비금액'].describe()

count    46969.000000
mean      2697.179883
std       2593.738706
min        -16.719125
25%        907.227250
50%       1687.343750
75%       3876.016750
max      14116.532250
Name: 소비금액, dtype: float64